# Photon–Dark Photon Entanglement — Full Cluster Analysis Notebook

In [ ]:
import os, sys, json
from pathlib import Path
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from astroquery.mast import Observations
from astropy.io import fits

REPO_ROOT = Path("/path/to/Primordial-Photon-Dark-Photon-Entanglement")
OUTPUT_ROOT = Path("Cluster_Analysis_Outputs")
OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)

CLUSTERS = ["Abell 1689"]
QUICK_MODE = False

sys.path.append(str(REPO_ROOT))
from Physics_Validation_Tests import run_validation
from Expected_Numerical_Results import compute_results

print('Environment ready.')


In [ ]:
def fetch_fits_for_target(target, out_dir):
    obs = Observations.query_criteria(target_name=target, obs_collection="HST")
    prods = Observations.get_product_list(obs)
    fits_prods = Observations.filter_products(prods, productType="SCIENCE", extension="fits")
    Observations.download_products(fits_prods, download_dir=out_dir)
    return list(Path(out_dir).glob("*.fits"))


In [ ]:
def process_fits_file(target, fits_file, target_dir):
    validation = run_validation(fits_file, quick=QUICK_MODE)
    numerical = compute_results(fits_file, quick=QUICK_MODE)
    return validation, numerical


In [ ]:
all_results = {}
for target in CLUSTERS:
    target_dir = OUTPUT_ROOT / target.replace(" ", "_")
    fits_dir = target_dir / "fits"
    fits_dir.mkdir(parents=True, exist_ok=True)
    fits_files = fetch_fits_for_target(target, fits_dir)
    target_results = []
    for f in fits_files:
        target_results.append(process_fits_file(target, f, target_dir))
    all_results[target] = target_results

all_results
